# Урок 4
1) Написать приложение, которое собирает основные новости с сайтов mail.ru и lenta.ru.<br>
2) (Дополнительно) Собрать все новости со страницы https://yandex.ru/news/<br>
<br>
Для парсинга использовать xpath. Структура данных должна содержать:<br>
* название источника,<br>
* наименование новости,<br>
* ссылку на новость,<br>
* дата публикации

In [390]:
import requests
import json
import pandas as pd
import time
import random
import sys
import locale
import warnings

from lxml import html
from pymongo import MongoClient
from requests.exceptions import HTTPError
from datetime import datetime

In [391]:
URL_MAIL = "https://m.mail.ru"
URL_LENTA = "https://lenta.ru"
URL_YANDEX = "https://yandex.ru/news"
USER_AGENT = "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36"

### Подключение к БД

In [392]:
def db_connect():
    return MongoClient('localhost', 27017)['news']

### Вывод статуса обработки

In [393]:
def show_status(text):
    sys.stdout.write(text)
    sys.stdout.flush()

### Получение HTML кода по ссылке

In [394]:
def get_html(url):
    try:
        response = requests.get(url = url, headers = {"User-Agent": USER_AGENT})
        if response.ok == True:
            return response.text
        else:
            response.raise_for_status()
    except Exception as err:
        print(f'Error: {err}')
        return ''

### Чтение HTML с возможностью кэширования

In [395]:
def get_html_cache(use_cache, cache, link):
    html = ''
    
    #Если считать данные из кеша не получилось или кэширование выключено...
    if use_cache:
        f = cache.find_one({"link": link})
        if f:
            html = f["html"]
    
    #...то достаём данные с сайта
    if not html:
        time.sleep(random.randint(1, 2))
        html = get_html(link)
        if html:
            cache.insert_one({"link": link, 
                              "html": html})
        
    return html

### Форматирование данных перед выводом

In [396]:
def format_data(data):
    result = []

    for d in data:
        result.append(['<a href="{}">{}</a>'.format(d['href'], d['text']),
                       d['date'],
                       '<a href="{}">{}</a>'.format(d['source'], d['source'])
                      ])
    return pd.DataFrame(result,
                        columns = ['href',
                                   'date',
                                   'source'
                                  ]
                       )

### Парсинг mail.ru

In [398]:
def parse_mail(use_cache, db, url, data):
    cnt = 0
    
    response = get_html_cache(use_cache, db.cache, url)
    if response:
        root = html.fromstring(response)

        #Блок новостей
        box_news = root.xpath('//div[contains(@class,"list") and contains(@id,"news-")]/a')
        #Цикл по новостям
        for news in box_news:
            new = {}
            new['href'] = news.xpath('string(./@href)')
            new['text'] = news.xpath('string(./*/span[@class="list__item__title"]/text())')
            new['date'] = ''
            new['source'] = url
            
            #Дату получаем проваливанием в страницу
            response = get_html_cache(use_cache, db.cache, new['href'])
            root_inner = html.fromstring(response)
            if root_inner:
                new['date'] = datetime.strptime(
                    root_inner.xpath('string(//meta[@property="article:published_time" or '
                                     '@name="article:published_time"]/@content)'),
                    "%Y-%m-%dT%H:%M:%S%z")
                    
            data.append(new)
            cnt += 1
            show_status(f"\rОбработано новостей {url}: {cnt}")
            
    show_status("\n")

### Парсинг lenta.ru

In [399]:
def parse_lenta(use_cache, db, url, data):
    locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')
    cnt = 0
    
    response = get_html_cache(use_cache, db.cache, url)
    if response:
        root = html.fromstring(response)

        #Блок новостей
        box_news = root.xpath('//section[@class="row b-top7-for-main js-top-seven"]/*/div[@class="item"]/a')
        #Цикл по новостям
        for news in box_news:
            new = {}
            new['href'] = news.xpath('string(./@href)')
            new['text'] = news.xpath('string(./text())')
            new['date'] = datetime.strptime(news.xpath('string(./time/@datetime)') + "+00:00", 
                                            " %H:%M, %d %B %Y%z")
            new['source'] = url

            data.append(new)
            cnt += 1
            show_status(f"\rОбработано новостей {url}: {cnt}")
    
    show_status("\n")

### Парсинг yandex.ru/news

In [400]:
def parse_yandex(use_cache, db, url, data):
    cnt = 0
    
    response = get_html_cache(use_cache, db.cache, url)
    if response:
        root = html.fromstring(response)

        #Блок новостей
        box_news = root.xpath('//div[contains(@class, "story story_view")]')
        #Цикл по новостям
        for news in box_news:
            new = {}

            link = news.xpath('.//a[@class="link link_theme_black i-bem"]')
            new['href'] = link[0].xpath('string(./@href)')
            new['text'] = link[0].xpath('string(./text())')
            new['date'] = datetime.strptime(datetime.today().strftime('%d.%m.%Y') + " " + 
                                            news.xpath('string(.//div[@class="story__date"]/text())')[-5::] + 
                                            ":00+00:00", '%d.%m.%Y %H:%M:%S%z')
            new['source'] = url

            data.append(new)
            cnt += 1
            show_status(f"\rОбработано новостей {url}: {cnt}")
    
    show_status("\n")

### Сбор новостей

In [403]:
warnings.filterwarnings("ignore")
data = []

#Приоритет чтения информации из кэша, а не с сайта
use_cache = True

#Подключаемся к БД
db = db_connect()

#Парсинг mail.ru
parse_mail(use_cache, db, URL_MAIL, data)

#Парсинг lenta.ru
parse_lenta(use_cache, db, URL_LENTA, data)

#Парсинг yandex.ru
parse_yandex(use_cache, db, URL_YANDEX, data)

#Форматирование данных
result = format_data(data)

result.style

Обработано новостей https://m.mail.ru: 32
Обработано новостей https://lenta.ru: 9
Обработано новостей https://yandex.ru/news: 65


,href,date,source
0,Украинские раскольники захотели изменить празднование Рождества,2019-11-24 22:20:06+03:00,https://m.mail.ru
1,Пилот рейса «Москва — Анапа» скончался после экстренной посадки,2019-11-24 12:01:22+03:00,https://m.mail.ru
2,«Детское Евровидение» выиграла исполнительница из Польши,2019-11-24 22:28:39+03:00,https://m.mail.ru
3,Умерла президент СПбГУ Людмила Вербицкая,2019-11-24 22:43:27+03:00,https://m.mail.ru
4,Россиян поразило внезапно обнажившееся дно Азовского моря,2019-11-24 08:59:25+03:00,https://m.mail.ru
5,Новый внедорожник для миллиардеров: он похож на битые «Жигули»,2019-11-22 17:30:45+03:00,https://m.mail.ru
6,"Культовая техника 90-х: вспоминаем, как это было (фото)",2019-11-23 08:29:00+03:00,https://m.mail.ru
7,«Какая красивая Аллочка»: Галкин показал новое фото Пугачевой,2019-11-23 13:11:43+03:00,https://m.mail.ru
8,Новый внедорожник для миллиардеров: он похож на битые «Жигули»,2019-11-22 17:30:45+03:00,https://m.mail.ru
9,Если бы медтест не отменили: бокал вина — и ты без прав,2019-11-24 09:59:11+03:00,https://m.mail.ru
